In [3]:
import os
import base64
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import pickle

# If modifying these SCOPES, delete the token.pickle file
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

def create_message(to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw}

def send_email(service, to, subject, body):
    message = create_message(to, subject, body)
    sent = service.users().messages().send(userId="me", body=message).execute()
    print(f"✅ Email sent to {to}, Message ID: {sent['id']}")

# === Usage ===
service = authenticate_gmail()

send_email(
    service,
    to="mrityunjairathore11d@gmail.com",
    subject="test mail",
    body="30 din mein 30 recruiters se 30 tareeke ke interview dilaunga."
)


✅ Email sent to mrityunjairathore11d@gmail.com, Message ID: 197ee56025ec8734


In [5]:
scheduled_email = {
    "to": "someone@example.com",
    "subject": "Scheduled Email Test",
    "body": "Hi! This email was scheduled using Gmail API and APScheduler.",
    "send_time": "2025-07-09 15:30"  # use a time 1–2 minutes in the future for testing
}


In [7]:
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta
import time
import os
import base64
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import pickle

# ========== Gmail Setup ==========
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

def create_message(to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw}

def send_email(service, to, subject, body):
    message = create_message(to, subject, body)
    sent = service.users().messages().send(userId="me", body=message).execute()
    print(f"✅ Email sent to {to}, Message ID: {sent['id']}")

# ========== Scheduling Setup ==========
def send_email_job():
    send_email(service, scheduled_email["to"], scheduled_email["subject"], scheduled_email["body"])

# Define your email
scheduled_email = {
    "to": "mrityunjairathore11d@gmail.com",
    "subject": "Scheduled Cold Email",
    "body": "Yeh email 30 din ke cold outreach campaign ka hissa hai. Automation FTW!",
    # Format: "YYYY-MM-DD HH:MM"
    "send_time": "2025-07-09 15:40"  # Update this time to be 1–2 mins ahead
}

# Convert send time to datetime object
send_time = datetime.strptime(scheduled_email["send_time"], "%Y-%m-%d %H:%M")

# Authenticate Gmail service
service = authenticate_gmail()

# Set up scheduler
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(send_email_job, 'date', run_date=send_time)

print(f"📅 Email scheduled for: {send_time.strftime('%Y-%m-%d %H:%M')}")

# Keep script running until email is sent
try:
    while True:
        time.sleep(2)
except (KeyboardInterrupt, SystemExit):
    scheduler.shutdown()


📅 Email scheduled for: 2025-07-09 15:40
